## Connect to KeyVault

In [1]:
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

keyVaultName = "gios-keyvault"
KVUri = f"https://gios-keyvault.vault.azure.net/"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

print(f"Retrieving your secrets from {keyVaultName}.")

os.environ["AZURE_OPENAI_API_KEY"] = client.get_secret("AZURE-OPENAI-API-KEY").value
os.environ["AZURE_OPENAI_ENDPOINT"] = client.get_secret("AZURE-OPENAI-ENDPOINT").value

os.environ["AZURE_SEARCH_API_KEY"] = client.get_secret("AZURE-SEARCH-API-KEY").value
os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"] = client.get_secret("AZURE-SEARCH-ENDPOINT").value

print("Done")

Retrieving your secrets from gios-keyvault.
Done


In [2]:
import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-002",
      "owner": "organization-owner",
      "id": "Davinci-002",
      "status": "succeeded",
      "created_at": 1677170437,
      "updated_at": 1677170437,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-ada-001",
      "owner": "organization-owner",
      "id": "ada-001",
      "status": "succeeded",
      "created_at": 1677170559,
      "updated_at": 1677170559,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-search-curie-doc-001",
      "owner": "organization-owner",
      "id": "text-search-curie-doc-001",
      "status": "succeeded",
      "created_at": 1677549363,
      "updated_at": 1677549363,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"


In [3]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Set the service endpoint and API key from the environment

service_name = "gios-cognitivesearch"#"YOUR-SEARCH-SERVICE-NAME"
admin_key = os.environ["AZURE_SEARCH_API_KEY"]#"YOUR-SEARCH-SERVICE-ADMIN-API-KEY"

index_name = "hotels-quickstart"

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)


search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

In [86]:
query="twin"

results =  search_client.search(search_text=query, include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())

SearchResults = ""

for result in results:
    print("{}: {}: {}".format(result["HotelId"], result["HotelName"] , result["Description"] ))
 
    SearchResults += result["HotelId"]+result["Description"]#+" \n " 

# prompt = '{\
#     "QUERY":'+query+', \
#     "SearchResults": '+SearchResults+'\
#     }Tl;dr'

prompt = SearchResults + "Tl;dr"
completion = openai.Completion.create(deployment_id="Davinci-002",
                                     prompt=prompt, stop="\n", temperature=0,
                                     echo=False,max_tokens= 100,
)
print("-----------------Result-----------------")                      
# print(f"{prompt}{completion['choices'][0]['text']}.")

print(f"{completion['choices'][0]['text']}.")


Total Documents Matching Query: 1
2: Twin Dome Motel: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.
-----------------Result-----------------
: The hotel is in an old plaza that's been renovated to be modern and comfortable, with some historical elements still present..


In [76]:
completion

<OpenAIObject text_completion id=cmpl-6p08uo0OwTDj2OtUMQyfkK07eRCr8 at 0x7fad8584d6c0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "2The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.Tl;dr: The hotel is in an old plaza that's been renovated to be modern and"
    }
  ],
  "created": 1677613296,
  "id": "cmpl-6p08uo0OwTDj2OtUMQyfkK07eRCr8",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 54,
    "total_tokens": 70
  }
}

In [6]:
def concatenate_values(dictionary, string):
    descriptions = [result[string] for result in dictionary]
    return " ".join(descriptions)

prompt = concatenate_values(results,"HotelName")
prompt

''

In [ ]:
summary = ""

#Get completions

Query = ""
SearchResults = ""



In [22]:
prompt = "The food was delicious and the waiter"
completion = openai.Completion.create(deployment_id="Davinci-002",
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")


The food was delicious and the waiter was very attentive.


In [58]:
prompt = "Say this is a test"
completion = openai.Completion.create(deployment_id="Davinci-002",
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")

Say this is a test

This is a test.
